In [1]:
#neural net
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch.optim as optim
import torch.nn.functional as F
import os

from CNN_small_architecture import CNNSmall
from CNN_layers import conv_homemade
from CNN_layers import maxpool_homemade
from CNN_layers import batchnorm_homemade, batchnorm_homemade_2

In [2]:
def tokenize(num):
    if num == 1:
        return torch.tensor(np.array([1., 0.]))
    else:
        return torch.tensor(np.array([0., 1.]))

MNIST_test = datasets.MNIST(root='./data', train=True, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]]

batch_size = 2
test_loader = DataLoader(test_set, batch_size=batch_size)

In [3]:
# takes a 4-dimensional tensor from the dataloader (batch_size x channel_size x height x width)
# and turns it into a list of lists of (height x width) numpy arrays
def transform_input(input_batch):
    return list(list(input_batch.detach().numpy()))

# compares two lists of batches with same amount of channels 
# and returns the summed absoute loss for each batch over the channels
def compare(x, y):
    b_diff = []
    batch_size = len(x)

    for b in range(batch_size):
        c_diff = []
        channels = len(x[b])

        for c in range(channels):
            diff = np.sum(np.absolute(x[b][c] - y[b][c]))
            c_diff.append(diff)
        
        b_diff.append(c_diff)
    
    return b_diff

In [4]:
model_original = CNNSmall()
model_original.eval()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

In [5]:
input_original_test, label_test = next(iter(test_loader))
print(np.array(input_original_test).shape)
input_homemade_test = transform_input(input_batch=input_original_test)

(2, 1, 28, 28)


In [6]:
def create_conv_homemade(model_conv):
    weights = model_conv.weight
    biases = model_conv.bias.detach().numpy()

    out_c, in_c, r, c = weights.shape
    conv1_filters = []

    for f in range(out_c):
        filter_ = []
        kernels = []

        for kernel in list(weights[f,:,:,:]):
            kernels.append(kernel.detach().numpy())

        filter_.append(kernels)
        filter_.append(biases[f])
        conv1_filters.append(filter_)
    
    return conv_homemade.Conv(filters=conv1_filters, in_channels=in_c)


In [7]:
def create_batchnorm_homemade(model_batchnorm):
    weights = model_batchnorm.weight
    biases = model_batchnorm.bias
    running_mean = model_batchnorm.running_mean
    running_var = model_batchnorm.running_var

    return batchnorm_homemade_2.BatchNorm(weights=weights, biases=biases, running_mean = running_mean, running_var = running_var)

In [8]:
def create_maxpool_homemade(model_maxpool):
    kernel_size = model_maxpool.kernel_size
    stride = model_maxpool.stride
    if type(model_maxpool.padding) == int:
        padding = (model_maxpool.padding, model_maxpool.padding)
    else:
        padding = model_maxpool.padding
    
    return maxpool_homemade.MaxPool(kernel_size=kernel_size, stride=stride, padding=padding)
    


In [9]:
conv1_homemade = create_conv_homemade(model_conv=model_original.conv1)
batchnorm1_homemade = create_batchnorm_homemade(model_batchnorm=model_original.batchNorm1)

In [10]:
# homemade conv1 filter on test
out_homemade = conv1_homemade(input_homemade_test)
# original conv1 filter on test
out_original = model_original.conv1(input_original_test)

print("Convolutional layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

Convolutional layer 1 error over out channels: 


[[4.042366603063119e-06, 3.4594775027430025e-06, 1.0153361123532711e-06],
 [6.455155075843488e-06, 5.694257553062743e-06, 1.924536561247192e-06]]

In [11]:
# homemade batchNorm1 filter on test
out_homemade = batchnorm1_homemade(input=out_homemade)
# original batchNorm1 filter on test
out_original = model_original.batchNorm1(out_original)

print("BatchNorm layer 1 error over out channels: ")
compare(out_homemade, transform_input(input_batch = out_original))

homemade:  [[-0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804  ]
 [-0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804  ]
 [-0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804   -0.359804   -0.359804   -0.359804   -0.359804
  -0.359804   -0.359804  ]
 [-0.359804   -0.359804   -0.359804   -0.359

[[20.581934231799096, 19.10204150690697, 30.70032871537842],
 [20.32747952779755, 27.384879199787974, 37.95119834225625]]